In [1]:
import matplotlib.pyplot as plt  # matplotlib version: 3.6.3
import numpy as np
import pandas as pd
import scipy.fft as fft
import plotly.graph_objects as go
import plotly.express as px
import webbrowser
from scipy.optimize import curve_fit
import os
import re
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [2]:
def esd(yt, fs):
    # computes ESD of a signal yt, keeping only positive frequencies! -> NOTE: this implies that total energy is halved
    N = len(yt)
    f = fft.rfftfreq(N, 1/fs)
    Sxx = np.square(1 / fs) *  np.square(np.abs(fft.rfft(yt)))
    
    return f, Sxx

In [3]:
dt = 0.005
fs_FTP = 10
k_subsample = int(1 / (dt * fs_FTP))

In [4]:
KPs = []
KI = 0.01
KDs = []

pattern_file = r'PID_([\d\.]+)_([\d\.]+)_([\d\.]+).csv'
pattern_folder = r'KP=([\d\.]+)'

points = []

z = []

for folder in os.listdir('Kx_exploration_data'):
    match_folder = re.match(pattern_folder, folder)
    KP = float(match_folder.group(1))
    KPs.append(KP)
    
    same_KP = []

    for file in os.listdir(os.path.join('Kx_exploration_data', folder)):
        match_file = re.match(pattern_file, file)
        if float(match_file.group(2)) == KI:
            KD = float(match_file.group(3))
                        
            filename = 'Kx_exploration_data' + os.sep + folder + os.sep + file
            signals_df = pd.read_csv(filename, sep=',', header=0)
            x_t = np.array(signals_df['Time'], dtype=float)
            y_rv = np.array(signals_df['Velocity'], dtype=float)
            y_tv = np.array(signals_df['Target Velocity'], dtype=float)
            y_tp = np.array(signals_df['Throttle'], dtype=float)

            x_t = x_t[::k_subsample]
            y_rv = y_rv[::k_subsample]
            y_tv = y_tv[::k_subsample]
            y_tp = y_tp[::k_subsample]
            
            f, Sxx = esd(y_tp, fs_FTP)

            AggIn = float(np.mean(Sxx) / np.std(y_tv - y_rv))

            # append KD value to KDs list, if the element is not already present
            if KD not in KDs:
                KDs.append(KD)  
            
            same_KP.append(AggIn)
            
            points.append({'KP': KP, 'KI' : KI, 'KD': KD, 'AggIn' : AggIn})
    
    z.append(same_KP)

In [5]:
def f(X, a, b, c, d, e, f, g, h, i, j):
    x1, x2 = X
    return a + b*x1 + c*x2 + d*x1*x2 + e*x1**2 + f*x2**2 + g*x1**3 + h*x2**3 + i*x1*x2**2 + j*x1**2*x2

In [6]:
in_mat = np.array([[d['KP'] for d in points], [d['KD'] for d in points]])

In [7]:
popt, pcov = curve_fit(f, xdata=in_mat, ydata=[d['AggIn'] for d in points])

In [8]:
pred_points = []

for d in points:
    pred_points.append({'KP': d['KP'], 'KD': d['KD'], 'AggIn' : f(np.array([d['KP'], d['KD']]), *popt)})

In [9]:
plot_folder = 'Kx_exploration_plots'

real_data = go.Surface(x = KPs, y = KDs, z = np.array(z).transpose(), colorscale = 'Viridis', name='Real data')
pred_data = go.Scatter3d(x = [d['KP'] for d in pred_points], y = [d['KD'] for d in pred_points], z = [d['AggIn'] for d in pred_points], 
                         mode = 'markers', marker = dict(size = 5, opacity = 0.8, color = 'red'), name='Predicted data')
fig = go.Figure(data = [real_data, pred_data])
fig.update_layout(  title = 'AggIn with fixed KI = 0.01',
                    scene=dict(xaxis_title='KP', yaxis_title='KD', zaxis_title='AggIn'))
fig.write_html(plot_folder + os.sep + 'real_vs_pred.html') 

webbrowser.open(plot_folder + os.sep + 'real_vs_pred.html',new=2)

True

In [10]:
pred_data = go.Surface(     x = np.linspace(0.04, 0.3, 100),
                            y = np.linspace(0.001, 1, 100), 
                            z = np.array([[f(np.array([KP, KD]), *popt) for KD in np.linspace(0.001, 1, 100)] for KP in np.linspace(0.04, 0.3, 100)]).transpose(), 
                            name='Predicted data',
                            opacity = 0.9)
real_data = go.Scatter3d(x = [d['KP'] for d in points], y = [d['KD'] for d in points], z = [d['AggIn'] for d in points], 
                         mode = 'markers', marker = dict(size = 5, opacity = 0.8, color = 'red'), name='Predicted data')

fig = go.Figure(data = [pred_data, real_data])
fig.update_layout(  title = 'AggIn with fixed KI = 0.01',
                    scene=dict(xaxis_title='KP', yaxis_title='KD', zaxis_title='AggIn'))
fig.write_html(plot_folder + os.sep + 'prediction_plot.html') 

webbrowser.open(plot_folder + os.sep + 'prediction_plot.html',new=2)

True

In [11]:
abs_errors = [np.abs(d['AggIn'] - f(np.array([d['KP'], d['KD']]), *popt)) for d in points]

In [12]:
np.mean(abs_errors)

1.4754651773375456

In [13]:
np.max(abs_errors)

9.158538612390387

In [14]:
np.std(abs_errors)

1.4285884068630734